# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 2:28PM,243586,21,622919,"NBTY Global, Inc.",Released
1,Aug 5 2022 2:25PM,243581,20,8365304,"ACI Healthcare USA, Inc.",Released
2,Aug 5 2022 2:23PM,243583,16,SEQIRUS-8356023,Seqirus,Released
3,Aug 5 2022 2:19PM,243584,16,TASA-277113,TASA USA Inc.,Released
4,Aug 5 2022 2:19PM,243585,21,622776-1,"NBTY Global, Inc.",Released
5,Aug 5 2022 2:17PM,243582,16,TASA-280052,TASA USA Inc.,Released
6,Aug 5 2022 2:09PM,243580,10,SOIVA0000510,Ivaoes Animal Health,Released
7,Aug 5 2022 2:09PM,243580,10,SOIVA0000512,Ivaoes Animal Health,Released
8,Aug 5 2022 2:09PM,243580,10,SOIVA0000511,Ivaoes Animal Health,Released
9,Aug 5 2022 2:02PM,243576,20,8355926,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,243582,Released,1
39,243583,Released,1
40,243584,Released,1
41,243585,Released,1
42,243586,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243582,NaN,NaN,1.0
243583,NaN,NaN,1.0
243584,NaN,NaN,1.0
243585,NaN,NaN,1.0
243586,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243497,0.0,1.0,0.0
243508,0.0,0.0,1.0
243509,0.0,0.0,1.0
243517,0.0,0.0,1.0
243518,2.0,14.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243497,0,1,0
243508,0,0,1
243509,0,0,1
243517,0,0,1
243518,2,14,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243497,0,1,0
1,243508,0,0,1
2,243509,0,0,1
3,243517,0,0,1
4,243518,2,14,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243497,,1,
1,243508,,,1
2,243509,,,1
3,243517,,,1
4,243518,2,14,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc."
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc."
2,Aug 5 2022 2:23PM,243583,16,Seqirus
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc."
5,Aug 5 2022 2:17PM,243582,16,TASA USA Inc.
6,Aug 5 2022 2:09PM,243580,10,Ivaoes Animal Health
9,Aug 5 2022 2:02PM,243576,20,"ACI Healthcare USA, Inc."
20,Aug 5 2022 2:02PM,243579,19,"GUSA Granules USA, Inc."
22,Aug 5 2022 1:49PM,243578,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc.",,,1
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc.",,,1
2,Aug 5 2022 2:23PM,243583,16,Seqirus,,,1
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.,,,1
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc.",,,1
5,Aug 5 2022 2:17PM,243582,16,TASA USA Inc.,,,1
6,Aug 5 2022 2:09PM,243580,10,Ivaoes Animal Health,,,3
7,Aug 5 2022 2:02PM,243576,20,"ACI Healthcare USA, Inc.",,,11
8,Aug 5 2022 2:02PM,243579,19,"GUSA Granules USA, Inc.",,,2
9,Aug 5 2022 1:49PM,243578,10,Beach Products Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc.",1,,
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 5 2022 2:23PM,243583,16,Seqirus,1,,
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.,1,,
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc.",1,,
5,Aug 5 2022 2:17PM,243582,16,TASA USA Inc.,1,,
6,Aug 5 2022 2:09PM,243580,10,Ivaoes Animal Health,3,,
7,Aug 5 2022 2:02PM,243576,20,"ACI Healthcare USA, Inc.",11,,
8,Aug 5 2022 2:02PM,243579,19,"GUSA Granules USA, Inc.",2,,
9,Aug 5 2022 1:49PM,243578,10,Beach Products Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc.",1,,
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 5 2022 2:23PM,243583,16,Seqirus,1,,
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.,1,,
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Aug 5 2022 2:23PM,243583,16,Seqirus,1.0,NaN,NaN
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.,1.0,NaN,NaN
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 2:28PM,243586,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Aug 5 2022 2:25PM,243581,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Aug 5 2022 2:23PM,243583,16,Seqirus,1.0,0.0,0.0
3,Aug 5 2022 2:19PM,243584,16,TASA USA Inc.,1.0,0.0,0.0
4,Aug 5 2022 2:19PM,243585,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2679183,42.0,23.0,2.0
12,243497,0.0,1.0,0.0
15,243509,1.0,0.0,0.0
16,2922576,9.0,3.0,0.0
19,1461360,7.0,8.0,4.0
20,730698,13.0,19.0,0.0
21,487171,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2679183,42.0,23.0,2.0
1,12,243497,0.0,1.0,0.0
2,15,243509,1.0,0.0,0.0
3,16,2922576,9.0,3.0,0.0
4,19,1461360,7.0,8.0,4.0
5,20,730698,13.0,19.0,0.0
6,21,487171,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,23.0,2.0
1,12,0.0,1.0,0.0
2,15,1.0,0.0,0.0
3,16,9.0,3.0,0.0
4,19,7.0,8.0,4.0
5,20,13.0,19.0,0.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,9.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,23.0,1.0,0.0,3.0,8.0,19.0,0.0
Released,42.0,0.0,1.0,9.0,7.0,13.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,23.0,1.0,0.0,3.0,8.0,19.0,0.0
2,Released,42.0,0.0,1.0,9.0,7.0,13.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,23.0,1.0,0.0,3.0,8.0,19.0,0.0
2,Released,42.0,0.0,1.0,9.0,7.0,13.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()